In [3]:
import secrets

random = secrets.SystemRandom()
print(random.randbytes(16).hex())

# start flash server with: python3 -m flask --app main run
# Note this only works when in the same dir.

#org_working_dir = os.getcwd()
#flask_working_dir = org_working_dir + "/local_test_server"
#os.chdir(flask_working_dir)
#instance = subprocess.Popen("python3 -m flask --app main run", shell=True)
#os.chdir(org_working_dir)

a89ff5a6313e5373b43607543c78929a


In [4]:
import requests


r = requests.get("http://localhost:5000/")
print(r.content)
print(r.cookies)

b'<p>Here, have a cookie!</p>'
<RequestsCookieJar[<Cookie authtoken=eec852cad7a6919eb7a2ee30837a2eb6a255348f41033a1e4c1bb1889825af28138225774459f3b880816ec1e3272923933e53371d6b51b6c4b7f2f22e2645076707f6515b19a797748fd12f8e8634d2dd543632c9aaf18334bb093dceb52948def5f7900f64f7ebcd193425c4f0c00b for localhost.local/>]>


In [5]:
from time import sleep
from requests import cookies

#Get authtoken:
r = requests.get("http://localhost:5000/")
authtoken: str = r.cookies["authtoken"]

#Manipulate authtoken:
authtoken_bytes = bytearray.fromhex(authtoken)
print(f'Token before: {authtoken_bytes.hex()}')
# We try to change the 8 first bytes of padding to something different
authtoken_bytes[0:16] = [0x6d for i in range(8)]#[authtoken_bytes[i] ^ 0x24 for i in range(8)]
print(f'Token after: {authtoken_bytes.hex()}')

# Request quote with the new token
cookie = {"authtoken": authtoken_bytes.hex()}
r = requests.get("http://localhost:5000/quote", cookies=cookie)
print(r.content)

# We see that we get a error msg instead that reveals that we are in CBC mode the token is padded 
# to 16 bytes.

Token before: bc2342d7ceb4d182ff5c4281d2ebf28501879b5ca30423e4d833d4fb5fec92848ce679f41608b4682b06219d4b0a1113fd1b5dd42ff47e51bce6fac4b47f3126b7c1d7f72c06909cb45a3136e7e11ea532ab8c81dcdc0811624e143eea7074de5ec1149941fa3b568b1072ff54d0374e
Token after: 6d6d6d6d6d6d6d6d01879b5ca30423e4d833d4fb5fec92848ce679f41608b4682b06219d4b0a1113fd1b5dd42ff47e51bce6fac4b47f3126b7c1d7f72c06909cb45a3136e7e11ea532ab8c81dcdc0811624e143eea7074de5ec1149941fa3b568b1072ff54d0374e
b'Data must be padded to 16 byte boundary in CBC mode'


In [9]:
#Manipulate authtoken:
authtoken_bytes = bytearray.fromhex(authtoken)
print(f'Token before: {authtoken_bytes.hex()}')
# We try to change the 8 first bytes of padding to something different
authtoken_bytes[0:16] = authtoken_bytes[16:32] #[authtoken_bytes[i] ^ 0x24 for i in range(8)]
print(f'Token after: {authtoken_bytes.hex()}')

# Request quote with the new token
cookie = {"authtoken": authtoken_bytes.hex()}
r = requests.get("http://localhost:5000/quote", cookies=cookie)
print(r.content)

# Note when changing the 16 padding bytes we get a different error message with a decode instead.

Token before: bc2342d7ceb4d182ff5c4281d2ebf28501879b5ca30423e4d833d4fb5fec92848ce679f41608b4682b06219d4b0a1113fd1b5dd42ff47e51bce6fac4b47f3126b7c1d7f72c06909cb45a3136e7e11ea532ab8c81dcdc0811624e143eea7074de5ec1149941fa3b568b1072ff54d0374e
Token after: 01879b5ca30423e4d833d4fb5fec928401879b5ca30423e4d833d4fb5fec92848ce679f41608b4682b06219d4b0a1113fd1b5dd42ff47e51bce6fac4b47f3126b7c1d7f72c06909cb45a3136e7e11ea532ab8c81dcdc0811624e143eea7074de5ec1149941fa3b568b1072ff54d0374e
b"'utf-8' codec can't decode byte 0xe4 in position 0: invalid continuation byte"
